In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [3]:
arbitrary_source = flux.catalog.obj_catalog[999]
arb_ra = arbitrary_source.ra_angle
arb_dec = arbitrary_source.dec_angle
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J.shape

(1, 2, 2)

In [6]:
Am = flux.stokes.A_matrix(arb_ra, arb_dec)
print(Am)
Am.shape

[[[ 7.33223331e-05+8.03131505e-05j -1.52572736e-04-3.57652091e-05j
    2.06862441e-05+2.56447185e-04j -3.68594242e-04+7.67424064e-05j]]

 [[-3.14619953e-04+1.60309398e-05j  4.97772743e-04-5.11341258e-04j
   -2.74206199e-04-3.26396462e-04j -1.62181054e-04+1.21515181e-03j]]

 [[-4.13518765e-04-4.65904433e-04j -2.04840599e-06+3.94961677e-04j
    4.23412575e-04+5.79906474e-04j -6.32822419e-05-1.07938943e-03j]]

 [[ 2.27657984e-04-8.17719079e-06j  2.72023955e-04-1.43638349e-04j
    9.87649746e-05-2.21315547e-04j  4.30672121e-04-1.48878366e-04j]]]


(4, 1, 4)

In [7]:
Am[0][0][0]

(7.332233306415914e-05+8.031315051463765e-05j)

In [10]:
# Display available antennae
print(flux.demo.active_ants)

[0, 1, 2, 11, 12, 13, 14, 23, 24, 25, 26, 27, 36, 37, 38, 39, 40, 41, 50, 51, 52, 53, 54, 55, 65, 66, 67, 68, 69, 70, 71, 82, 83, 84, 85, 86, 87, 88, 98, 120, 121, 122, 123, 124, 136, 137, 138, 139, 140, 141, 142, 143]


In [20]:
# we sample three sources almost at random, and 2/3 are zero

print(arbitrary_source)
print(flux.vis.visibility(2, 11, arbitrary_source))

print(flux.vis.visibility(36, 37, flux.catalog.obj_catalog[981]))
print(flux.vis.visibility(88, 121, flux.catalog.obj_catalog[134]))
# also, would we not expect the first term to dominate? And yet it is not even the largest
    # of numbers all with magnitudes less than one

Name: GLEAM J185915-381842
Right ascension: 284.8155
Declination: -37.688250000000004
151 MHz flux: 1.2316479999999999

[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[ 1.06831717e-02-1.46480343e-02j]
 [-4.75270222e-01+4.86890844e-03j]
 [ 2.49743053e-02+4.73241619e-01j]
 [-2.05405352e-04+3.26876788e-04j]]


In [41]:
# proposal for version 7: change J_matrix to include local sidereal time

def J_matrix_7(ra, dec, lst=None, nu=150e6):
    """
    Return the Jones matrix J.
    @ra: right ascension of the source, in degrees.
    @dec: right ascension of the source, in degrees.
    @nu: frequency of interest, in Hz.

    The default argument comes from the beam that I
    had access to when this was written.
    """
    if lst is None:
        lst = rot.get_lst()
    latitude = np.radians(flux.rot.hera_lat)
    # We want to transform the following into a function parameter
    az, alt = flux.rot.eq_to_topo(ra, dec, latitude, lst, radians=True)

    az = np.array([az])
    alt = np.array([alt])
    
    return flux.stokes.spline_beam_func(nu, alt, az)

def A_matrix_7(ra, dec, nu=150e6):
    """
    Return the Mueller matrix A.
    @ra: right ascension of the source, in degrees.
    @dec: declination of the source, in degrees.
    @nu: frequency of interest, in Hz.

    The default argument comes from the beam that I
    had access to when this was written.
    """
    J = J_matrix_7(ra, dec, nu)
    assert J is not None
    J_outer = np.kron(J, np.conj(J))
    return np.dot(flux.stokes.S, np.dot(J_outer, np.linalg.inv(flux.stokes.S)))
    
def visibility_7(ant1, ant2, source, nu=151e6):
    """
    Visibility integrand evaluated for a single source.

    The most glaring waste of compute is separately calculating
    the values for RA and DEC, although it is not clear to me
    how many clock cycles are actually spent thereon.
    """
    I = source.flux_by_frq[nu / 1e6]
    s = np.array([complex(I), 0, 0, 0])

    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    r = flux.rot.raddec2lm(ra, dec)

    phi = flux.ant.phase_factor(ant1, ant2, r, nu)
    return np.dot(np.dot(A_matrix_7(ra, dec, nu), s), phi)

In [46]:
t = 0
while t < 2 * np.pi:
    print(t)
    t += np.pi / 72
    print(J_matrix_7(88, 121, t, flux.catalog.obj_catalog[134]))
    print("\n")


0


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
[1] During: typing of argument at //anaconda3/envs/hera/lib/python3.7/site-packages/RIMEz/beam_models.py (110)

File "../../../../../../../anaconda3/envs/hera/lib/python3.7/site-packages/RIMEz/beam_models.py", line 110:
    def spline_beam_func(nu, alt, az):
        i_nu = np.where(nu == nu_axis)[0][0]
        ^

This error may have been caused by the following argument(s):
- argument 0: cannot determine Numba type of <class 'flux.catalog.GLEAM_entry'>
